In [1]:
import itertools

import numpy as np

import ambulance_game as abg

## Closed form waiting time formula

In [2]:
lambda_1 = 1
lambda_2 = 1
mu = 3
num_of_servers = 1
threshold = 2
system_capacity = 3
buffer_capacity = 3

In [3]:
all_states = abg.markov.build_states(
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
transition_matrix = abg.markov.get_transition_matrix(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
steady_state_vector = abg.markov.get_steady_state_algebraically(
    Q=transition_matrix, algebraic_function=np.linalg.solve
)
pi = abg.markov.get_markov_state_probabilities(
    pi=steady_state_vector, all_states=all_states, output=np.ndarray
)

In [4]:
abg.markov.mean_waiting_time_formula_using_closed_form_approach(
    all_states=all_states,
    pi=pi,
    class_type=0,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)

0.3000989958987412

## Direct waiting time formula

In [5]:
def get_coefficients_row_of_array_for_state(
    state, 
    class_type,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity
):
    lhs_coefficient_row = np.zeros([buffer_capacity + 1, system_capacity + 1])
    lhs_coefficient_row[state[0], state[1]] = -1
    for (u, v) in itertools.product(range(1, buffer_capacity + 1), range(threshold)):
        lhs_coefficient_row[u, v] = np.NaN

    rhs_value = 0
    if abg.markov.is_waiting_state(state, num_of_servers):
        if state[0] >= 1 and state[1] == threshold:
            next_state = (state[0] - 1, state[1])
        else:
            next_state = (state[0], state[1] - 1)
        
        lhs_coefficient_row[next_state[0], next_state[1]] = 1
        rhs_value = -abg.markov.expected_time_in_markov_state_ignoring_arrivals(
            state=state,
            class_type=class_type,
            mu=mu,
            num_of_servers=num_of_servers,
            threshold=threshold,
        )

    vectorised_array = np.hstack(
        (
            lhs_coefficient_row[0, :threshold], 
            lhs_coefficient_row[:, threshold:].flatten("F")
        )
    )
    return vectorised_array, rhs_value

In [6]:
def get_waiting_time_linear_system(
    class_type, mu, num_of_servers, threshold, system_capacity, buffer_capacity
):
    all_coefficients_array = np.array([])
    all_states = abg.markov.build_states(
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    for state in all_states:
        lhs_vector, rhs_value = get_coefficients_row_of_array_for_state(
            state=state, 
            class_type=class_type,
            mu=mu,
            num_of_servers=num_of_servers,
            threshold=threshold,
            system_capacity=system_capacity,
            buffer_capacity=buffer_capacity
        )
        if len(all_coefficients_array) == 0:
            all_coefficients_array = [lhs_vector]
            constant_column = [rhs_value]
        else:
            all_coefficients_array = np.vstack(
                [all_coefficients_array, lhs_vector]
            )
            constant_column.append(rhs_value)
    return all_coefficients_array, constant_column

In [7]:
def convert_solution_to_correct_array_format(
    array, all_states, system_capacity, buffer_capacity
):
    array_with_correct_shape = np.zeros([buffer_capacity + 1, system_capacity + 1])
    for index, (u, v) in enumerate(all_states):
        array_with_correct_shape[u, v] = array[index]
    return array_with_correct_shape

In [8]:
def get_waiting_times_of_all_states_using_direct_approach(
    class_type, all_states, mu, num_of_servers, threshold, system_capacity, buffer_capacity
):
    M, b = get_waiting_time_linear_system(
        class_type, mu, num_of_servers, threshold, system_capacity, buffer_capacity
    )
    state_waiting_times = np.linalg.solve(M, b)
    state_waiting_times = convert_solution_to_correct_array_format(
        state_waiting_times, all_states, system_capacity, buffer_capacity
    )
    return state_waiting_times

In [9]:
def mean_waiting_time_formula_using_direct_approach(
    all_states,
    pi,
    class_type,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity,
    **kwargs
):
    M, b = get_waiting_time_linear_system(
        class_type=class_type,
        mu=mu,
        num_of_servers=num_of_servers,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    sol = np.linalg.solve(M, b)
    waiting_times = convert_solution_to_correct_array_format(
       sol, all_states, system_capacity, buffer_capacity
    )

    mean_waiting_time, prob_accept_class_2_ind = 0, 0
    for index, (u, v) in enumerate(all_states):
        if abg.markov.is_accepting_state(
            state=(u, v),
            class_type=class_type,
            threshold=threshold,
            system_capacity=system_capacity,
            buffer_capacity=buffer_capacity,
        ):
            arriving_state = (u, v + 1)
            if class_type == 1 and v >= threshold:
                arriving_state = (u + 1, v)
            mean_waiting_time += waiting_times[arriving_state] * pi[u, v]
            prob_accept_class_2_ind += pi[u, v]
        
    return mean_waiting_time / prob_accept_class_2_ind    

In [10]:
lambda_1 = 1
lambda_2 = 1
mu = 3
num_of_servers = 1
threshold = 2
system_capacity = 3
buffer_capacity = 3

all_states = abg.markov.build_states(
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
transition_matrix = abg.markov.get_transition_matrix(
    lambda_2=lambda_2,
    lambda_1=lambda_1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
steady_state_vector = abg.markov.get_steady_state_algebraically(
    Q=transition_matrix, algebraic_function=np.linalg.solve
)
pi = abg.markov.get_markov_state_probabilities(
    pi=steady_state_vector, all_states=all_states, output=np.ndarray
)
abg.markov.mean_waiting_time_formula_using_closed_form_approach(
    all_states=all_states,
    pi=pi,
    class_type=1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)

0.20552268244575936

In [11]:
mean_waiting_time_formula_using_direct_approach(
    all_states=all_states,
    pi=pi,
    class_type=1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)

0.20552268244575936

In [13]:
for parameter_1 in np.linspace(0.2, 0.2, 1):
    for parameter_2 in np.linspace(0.2, 0.2, 1):
        for parameter_3 in np.linspace(0.2, 30, 10):
            for parameter_4 in np.linspace(1, 10, 10, dtype=int):
                res_1 = abg.markov.get_mean_waiting_time_using_markov_state_probabilities(
                    lambda_2=parameter_1,
                    lambda_1=parameter_2,
                    mu=parameter_3,
                    num_of_servers=parameter_4,
                    threshold=threshold,
                    system_capacity=20,
                    buffer_capacity=15,
                    class_type=1,
                    waiting_formula=mean_waiting_time_formula_using_direct_approach,
                )

                res_2 = abg.markov.get_mean_waiting_time_using_markov_state_probabilities(
                    lambda_2=parameter_1,
                    lambda_1=parameter_2,
                    mu=parameter_3,
                    num_of_servers=parameter_4,
                    threshold=threshold,
                    system_capacity=20,
                    buffer_capacity=15,
                    class_type=1,
                )
                assert np.isclose(res_1, res_2)

In [18]:
all_states = [
    (0, 0),
    (0, 1),
    (0, 2),
    (0, 3),
    (1, 3),
    (2, 3),
    (3, 3),
]
pi = np.array(
    [
        [0.11428571, 0.22857143, 0.22857143, 0.22857143],
        [np.nan, np.nan, np.nan, 0.11428571],
        [np.nan, np.nan, np.nan, 0.05714286],
        [np.nan, np.nan, np.nan, 0.02857143],
    ]
)

mean_times = [
    mean_waiting_time_formula_using_direct_approach(
        all_states=all_states,
        pi=pi,
        class_type=class_type,
        lambda_1=1,
        lambda_2=1,
        mu=1,
        num_of_servers=2,
        threshold=3,
        system_capacity=3,
        buffer_capacity=3,
    )
    for class_type in range(2)
]
mean_times

[0.20000000175, 0.32352941297577853]

## Playground

In [52]:
def is_waiting_state(state, num_of_servers):
    return state[1] > num_of_servers

In [53]:
def expected_time_in_markov_state_ignoring_arrivals(
    state,
    class_type,
    mu,
    num_of_servers,
    threshold,
):
    if state[0] > 0 and (state[1] == threshold or class_type == 1):
        return 0
    return 1 / (min(state[1], num_of_servers) * mu)

In [54]:
def get_coefficients_row_of_array_for_state(
    state, 
    class_type,
    mu,
    num_of_servers,
    threshold,
    system_capacity,
    buffer_capacity
):
    lhs_coefficient_row = np.zeros([buffer_capacity + 1, system_capacity + 1])
    lhs_coefficient_row[state[0], state[1]] = -1
    for (u, v) in itertools.product(range(1, buffer_capacity + 1), range(threshold)):
        lhs_coefficient_row[u, v] = np.NaN

    rhs_value = 0
    if is_waiting_state(state, num_of_servers):
        if state[0] >= 1 and state[1] == threshold:
            next_state = (state[0] - 1, state[1])
        else:
            next_state = (state[0], state[1] - 1)

        
        lhs_coefficient_row[next_state[0], next_state[1]] = 1

        rhs_value = -expected_time_in_markov_state_ignoring_arrivals(
            state=state,
            class_type=class_type,
            mu=mu,
            num_of_servers=num_of_servers,
            threshold=threshold,
        )

    vectorised_array = np.hstack(
        (
            lhs_coefficient_row[0, :threshold], 
            lhs_coefficient_row[:, threshold:].flatten("F")
        )
    )
    return vectorised_array, rhs_value

In [55]:
for state in all_states:
    print(state, get_coefficients_row_of_array_for_state(
        state=state,
        class_type=0,
        mu=mu,
        num_of_servers=num_of_servers,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    ))

(0, 0) (array([-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0)
(0, 1) (array([ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0)
(0, 2) (array([ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), -0.3333333333333333)
(1, 2) (array([ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.]), 0)
(2, 2) (array([ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.]), 0)
(3, 2) (array([ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.]), 0)
(0, 3) (array([ 0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.]), -0.3333333333333333)
(1, 3) (array([ 0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.]), -0.3333333333333333)
(2, 3) (array([ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.]), -0.3333333333333333)
(3, 3) (array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.]), -0.3333333333333333)


In [60]:
def get_waiting_time_linear_system(
    class_type, mu, num_of_servers, threshold, system_capacity, buffer_capacity
):
    all_coefficients_array = np.array([])
    all_states = abg.markov.build_states(
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    )
    for state in all_states:
        lhs_vector, rhs_value = get_coefficients_row_of_array_for_state(
            state=state, 
            class_type=class_type,
            mu=mu,
            num_of_servers=num_of_servers,
            threshold=threshold,
            system_capacity=system_capacity,
            buffer_capacity=buffer_capacity
        )
        if len(all_coefficients_array) == 0:
            all_coefficients_array = [lhs_vector]
            constant_column = [rhs_value]
        else:
            all_coefficients_array = np.vstack(
                [all_coefficients_array, lhs_vector]
            )
            constant_column.append(rhs_value)
    return all_coefficients_array, constant_column

In [61]:
M, b = get_waiting_time_linear_system(
    class_type=1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)
M

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0., -1.]])

In [62]:
sol = np.linalg.solve(M, b)
sol

array([-0.        , -0.        ,  0.33333333,  0.33333333,  0.33333333,
        0.33333333,  0.66666667,  0.33333333,  0.33333333,  0.33333333])

In [63]:
def convert_solution_to_correct_array_format(
    array, all_states, system_capacity, buffer_capacity
):
    array_with_correct_shape = np.zeros([buffer_capacity + 1, system_capacity + 1])
    for index, (u, v) in enumerate(all_states):
        array_with_correct_shape[u, v] = sol[index]
    return array_with_correct_shape

In [64]:
waiting_times = convert_solution_to_correct_array_format(
    sol, all_states, system_capacity, buffer_capacity
)

In [65]:
mean_blocking_time, prob_accept_class_2_ind = 0, 0
for index, (u, v) in enumerate(all_states):
    if abg.markov.is_accepting_state(
        state=(u, v),
        class_type=1,
        threshold=threshold,
        system_capacity=system_capacity,
        buffer_capacity=buffer_capacity,
    ):
        arriving_state = (u + 1, v) if v >= threshold else (u, v + 1)
        mean_blocking_time += waiting_times[arriving_state] * pi[u, v]
        prob_accept_class_2_ind += pi[u, v]
    
mean_blocking_time / prob_accept_class_2_ind

0.20552268244575936

In [19]:
abg.markov.mean_waiting_time_formula_using_closed_form_approach(
    all_states=all_states,
    pi=pi,
    class_type=1,
    mu=mu,
    num_of_servers=num_of_servers,
    threshold=threshold,
    system_capacity=system_capacity,
    buffer_capacity=buffer_capacity,
)

0.20552268244575936